In [ ]:
## Imports
import random
random.seed(10)
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

In [ ]:
## DataLoader
class CarSegData(Dataset):
    def __init__(self, data_root, transform=None):
        self.data_root = data_root
        self.transform = transform
        self.class_labels = {
            10: 1,
            20: 2,
            30: 3,
            40: 4,
            50: 5,
            60: 6,
            70: 7,
            80: 8,
            90: 0
        }
        self.classes = {
            1: "hood",
            2: "front door",
            3: "rear door",
            4: "frame",
            5: "rear quarter panel",
            6: "trunk lid",
            7: "fender",
            8: "bumper",
            9: "rest of car"
        }

        # List all the array files in the 'arrays' directory
        self.array_files = np.load(data_root)

    def __len__(self):
        return np.shape(self.array_files)[0]

    def __getitem__(self, idx):
        array_data = self.array_files[idx,:,:,:]
        image_data = array_data[:,:,:3]
        target_data = array_data[:,:,3]

        # Convert target data to class labels
        target_data = self.map_to_class_labels(target_data)
        target_data = self.map_to_classes(target_data)

        # Convert to PIL image
        image = Image.fromarray(image_data.astype('uint8'))

        if self.transform:
            image = self.transform(image)

        return image, target_data

    def map_to_classes(self, target_data):
        class_labels = np.zeros_like(target_data)
        for class_value, class_name in self.classes.items():
            class_labels[target_data == class_value] = class_value
        return torch.from_numpy(class_labels)
    
    def map_to_class_labels(self, target_data):
        class_labels = np.zeros_like(target_data)
        for old_label, new_label in self.class_labels.items():
            class_labels[target_data == old_label] = new_label
        return torch.from_numpy(class_labels)

In [ ]:
# Unet 
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d()
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet_new(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet_new, self).__init__()

        # Encoder (contracting path)
        self.enc1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Middle layer
        self.middle = DoubleConv(512, 1024)
#         self.dropout = nn.Dropout2d(p=0.5)

        # Decoder (expansive path)
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(128, 64)
        
        # Output layer
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        pool1 = self.pool1(enc1)
        enc2 = self.enc2(pool1)
        pool2 = self.pool2(enc2)
        enc3 = self.enc3(pool2)
        pool3 = self.pool3(enc3)
        enc4 = self.enc4(pool3)
        pool4 = self.pool4(enc4)

        # Middle layer
        middle = self.middle(pool4)
#         middle = self.dropout(middle)

        # Decoder with skip connections
        up1 = self.up1(middle)
        concat1 = torch.cat((up1, enc4), dim=1)
        dec1 = self.dec1(concat1)
        up2 = self.up2(dec1)
        concat2 = torch.cat((up2, enc3), dim=1)
        dec2 = self.dec2(concat2)
        up3 = self.up3(dec2)
        concat3 = torch.cat((up3, enc2), dim=1)
        dec3 = self.dec3(concat3)
        up4 = self.up4(dec3)
        concat4 = torch.cat((up4, enc1), dim=1)
        dec4 = self.dec4(concat4)
        
        # Output layer
        out = self.out(dec4)
        return out

In [ ]:
## TRAINING
# Move the model and data to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f'Training on {device}')
# Define the input and output channels
in_channels = 3  # Input channels (RGB)
out_channels = 9  # Number of classes

# Create the U-Net model
model = UNet_new(in_channels, out_channels)
model.to(device)

# path to data
path = '/scratch/s204254/CarSegData/carseg_data_new_new/only_real.npy'
transform = transforms.Compose([
    transforms.ToTensor()# Converts the image to a tensor
])


dataset = CarSegData(data_root=path,transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [130, 8])


# Define batch size and other DataLoader parameters
batch_size = 32
num_workers = 2
learning_rate = 0.001
num_epochs = 2000
wd = 1e-4
dropout = 0.5
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay = wd)
criterion = nn.CrossEntropyLoss()

# Create the data loader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(f"Number of batches in training set: {len(train_loader)}")
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(f"Number of batches in validation set: {len(val_loader)}")

# Training loop
print("~  B E G I N   T R A I N I N G  ~\n")
validation_every_number_of_epoch = 1
train_loss_all = []
val_loss_all = []
best_score = 1000000
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    train_loss = []

    for i, batch in enumerate(train_loader):
        images, labels = batch
        images, labels = images.to(device), labels.to(device).type(torch.long)

        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        train_loss.append(loss.item())
    if epoch % validation_every_number_of_epoch == 0:
        val_loss = []
        with torch.no_grad():
            model.eval()
            for i, batch in enumerate(val_loader):
                images, labels = batch
                images, labels = images.to(device), labels.to(device).type(torch.long)

                # Forward pass
                outputs = model(images)

                # Compute the loss
                loss = criterion(outputs, labels)
                val_loss.append(loss.item())
    model.train()

    
    sum_loss = np.sum(train_loss)
    n_losses = len(train_loss)
    print(f"Epoch {epoch+1} average train loss: {sum_loss/n_losses:.4f}")
    train_loss_all.append(sum_loss/n_losses)
    
    sum_val_loss = np.sum(val_loss)
    n_val_losses = len(val_loss)
    print(f"Epoch {epoch+1} average validation loss: {sum_val_loss/n_val_losses:.4f} ")
    val_loss_all.append(sum_val_loss/n_val_losses)
print(" ")    
print("~ F I N I S H E D   T R A I N I N G ~")